# Smart Gift Planner

### Objective

Create and design a High-Fidelity Mobile App Prototype for a Smart Gift Planner. The prototype must demonstrate the user flow from defining a gift recipient to viewing the personalized, algorithmic recommendations.

## Import Libraries and Data

### Import Libraries

In [3]:
# Dataframes
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)

# AI
from google import genai
from dotenv import load_dotenv
import time
import os
import math
import ast

# Visualizations
import plotly.express as px

### Import Data

In [4]:
products = pd.read_csv("data/amazon_products.csv")
categories = pd.read_csv("data/amazon_categories.csv")

In [5]:
products.head()

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.50,0,139.99,0.00,104,False,2000
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.50,0,169.99,209.99,104,False,1000
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,https://www.amazon.com/dp/B07XSCCZYG,4.60,0,365.49,429.99,104,False,300
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,https://www.amazon.com/dp/B08MVFKGJM,4.60,0,291.59,354.37,104,False,400
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,https://www.amazon.com/dp/B01DJLKZBA,4.50,0,174.99,309.99,104,False,400


In [6]:
categories.head()

,id,category_name
0,1,Beading & Jewelry Making
1,2,Fabric Decorating
2,3,Knitting & Crochet Supplies
3,4,Printmaking Supplies
4,5,Scrapbooking & Stamping Supplies


In [7]:
# Merge the 2 dfs on category ID
merged = products.merge(
    categories,
    left_on="category_id",
    right_on="id",
    how="left"
)

# Remove numerical category identifiers
merged = merged.drop(columns=["id", "category_id"])

# Save to new .csv
merged.to_csv("data/merged_products.csv", index=False, encoding="utf-8")

# Save to JSON for SE/UI/UX
merged.to_json("data/merged_products.json", orient="records", lines=True)

merged.head()

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,isBestSeller,boughtInLastMonth,category_name
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.50,0,139.99,0.00,False,2000,Suitcases
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.50,0,169.99,209.99,False,1000,Suitcases
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,https://www.amazon.com/dp/B07XSCCZYG,4.60,0,365.49,429.99,False,300,Suitcases
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,https://www.amazon.com/dp/B08MVFKGJM,4.60,0,291.59,354.37,False,400,Suitcases
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,https://www.amazon.com/dp/B01DJLKZBA,4.50,0,174.99,309.99,False,400,Suitcases


## Exploratory Data Analysis

In [8]:
merged.info()
print(merged.isna().sum())
merged.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1426337 entries, 0 to 1426336
Data columns (total 11 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   asin               1426337 non-null  object 
 1   title              1426336 non-null  object 
 2   imgUrl             1426337 non-null  object 
 3   productURL         1426337 non-null  object 
 4   stars              1426337 non-null  float64
 5   reviews            1426337 non-null  int64  
 6   price              1426337 non-null  float64
 7   listPrice          1426337 non-null  float64
 8   isBestSeller       1426337 non-null  bool   
 9   boughtInLastMonth  1426337 non-null  int64  
 10  category_name      1426337 non-null  object 
dtypes: bool(1), float64(3), int64(2), object(5)
memory usage: 110.2+ MB
asin                 0
title                1
imgUrl               0
productURL           0
stars                0
reviews              0
price                0
lis

,stars,reviews,price,listPrice,boughtInLastMonth
count,1426337.00,1426337.00,1426337.00,1426337.00,1426337.00
mean,4.00,180.75,43.38,12.45,141.98
std,1.34,1761.45,130.29,46.11,836.27
min,0.00,0.00,0.00,0.00,0.00
25%,4.10,0.00,11.99,0.00,0.00
50%,4.40,0.00,19.95,0.00,0.00
75%,4.60,0.00,35.99,0.00,50.00
max,5.00,346563.00,19731.81,999.99,100000.00


In [9]:
# How many products have reviews?
print((merged['reviews'] != 0).sum())

# How many products are best sellers?
print((merged['isBestSeller'] == True).sum())

295834
8520


In [10]:
# Drop row with missing product title
merged = merged.dropna(subset=['title'])

In [11]:
# Check for unique product categories before using AI to categorize them
print(merged['category_name'].unique())
print(len(merged['category_name'].unique()))

['Suitcases' "Men's Clothing" 'Xbox 360 Games, Consoles & Accessories'
 "Men's Shoes" "Men's Accessories" 'Vacuum Cleaners & Floor Care'
 'Televisions & Video Products' 'Additive Manufacturing Products'
 'Headphones & Earbuds' 'PlayStation Vita Games, Consoles & Accessories'
 'Wii U Games, Consoles & Accessories'
 'PlayStation 4 Games, Consoles & Accessories' "Boys' Watches"
 "Girls' Clothing" "Boys' Clothing" 'Pregnancy & Maternity Products'
 'Shaving & Hair Removal Products' 'Fabric Decorating'
 'Industrial Materials' 'Smart Home: Security Cameras and Systems'
 'Office Electronics' 'Sports & Outdoor Play Toys' "Kids' Play Tractors"
 'Slot Cars, Race Tracks & Accessories' 'Video Games'
 'Smart Home: Voice Assistants and Hubs' 'Light Bulbs' 'Toys & Games'
 "Kids' Furniture" 'Automotive Tires & Wheels'
 'Wellness & Relaxation Products' 'Automotive Tools & Equipment'
 'Baby & Toddler Toys' "Kids' Play Boats" 'Computer Monitors'
 "Girls' Jewelry" 'Luggage' 'Printmaking Supplies' "Women's 

The merged df only contains products from 248 out of a total of 270 possible product categories. Regardless, I will categorize all of them with Gemini.

## Data Preprocessing

### Broad Product Categorization (Gemini)

This isn't really needed, I used ChatGPT below. At first I was going to run all products but the category is enough.

In [12]:
"""
# -----------------------------
# Setup client
# -----------------------------
load_dotenv()
api_key_env = os.getenv("API_KEY")  # from local environment variable
client = genai.Client(api_key=api_key_env)

MODEL_NAME = "gemini-2.5-flash"

# -----------------------------
# Free tier batching parameters
# -----------------------------
batch_size = 5                # number of articles per batch (≤ free tier limit)
sleep_time = 35               # seconds to sleep between batches

# List of 270 Amazon categories
amazon_categories = merged['category_name'].unique()
total_categories = len(amazon_categories)
total_batches = (total_categories + batch_size - 1) // batch_size

# Function to categorize into broader categories
def categorize_batch(categories):
    prompt = f"""
#You are an AI assistant. I have a list of Amazon product categories. 
#Please categorize each of the following categories into a broader, high-level category such as 'Electronics', 'Clothing', 'Home', 'Beauty', 'Food', 'Sports', 'Toys', etc. 

#Return the result as a Python dictionary where keys are the original categories and values are the broad category.
#Categories:
#{categories}
"""
    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=prompt
    )
    
# Access the text
    text_response = response.text.strip()
    
    cleaned_response = (
    text_response
    .replace("```python", "")
    .replace("```", "")
    .strip()
)
    
    print(cleaned_response)
    try:
        return ast.literal_eval(text_response)  # convert string dict to Python dict
    except:
        print("Error parsing batch, returning raw text")
        return text_response

# Loop over batches
all_results = {}
for i in range(total_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, total_categories)
    batch = amazon_categories[start_idx:end_idx].tolist()
    
    print(f"Processing batch {i+1}/{total_batches}...")
    batch_result = categorize_batch(batch)
    
    if isinstance(batch_result, dict):
        all_results.update(batch_result)
    else:
        print(f"Batch {i+1} returned invalid format, saving as text")
        all_results[f"batch_{i+1}"] = batch_result
    
    if i < total_batches - 1:
        time.sleep(sleep_time)  # avoid exceeding free-tier rate limit

# Final results
print("Categorization complete!")
print(all_results)

# Use all_results to create a new df column
merged['broad_category'] = merged['category_name'].map(all_results)
"""

'\n    response = client.models.generate_content(\n        model=MODEL_NAME,\n        contents=prompt\n    )\n\n# Access the text\n    text_response = response.text.strip()\n\n    cleaned_response = (\n    text_response\n    .replace("```python", "")\n    .replace("```", "")\n    .strip()\n)\n\n    print(cleaned_response)\n    try:\n        return ast.literal_eval(text_response)  # convert string dict to Python dict\n    except:\n        print("Error parsing batch, returning raw text")\n        return text_response\n\n# Loop over batches\nall_results = {}\nfor i in range(total_batches):\n    start_idx = i * batch_size\n    end_idx = min(start_idx + batch_size, total_categories)\n    batch = amazon_categories[start_idx:end_idx].tolist()\n\n    print(f"Processing batch {i+1}/{total_batches}...")\n    batch_result = categorize_batch(batch)\n\n    if isinstance(batch_result, dict):\n        all_results.update(batch_result)\n    else:\n        print(f"Batch {i+1} returned invalid format, sa

### Broad Product Categorization (ChatGPT)

In [13]:
categories_dict = {
    'Suitcases': 'Travel',
    "Men's Clothing": 'Clothing',
    'Xbox 360 Games, Consoles & Accessories': 'Electronics',
    "Men's Shoes": 'Clothing',
    "Men's Accessories": 'Clothing',
    'Vacuum Cleaners & Floor Care': 'Home',
    'Televisions & Video Products': 'Electronics',
    'Additive Manufacturing Products': 'Industrial',
    'Headphones & Earbuds': 'Electronics',
    'PlayStation Vita Games, Consoles & Accessories': 'Electronics',
    'Wii U Games, Consoles & Accessories': 'Electronics',
    'PlayStation 4 Games, Consoles & Accessories': 'Electronics',
    "Boys' Watches": 'Clothing',
    "Girls' Clothing": 'Clothing',
    "Boys' Clothing": 'Clothing',
    'Pregnancy & Maternity Products': 'Baby',
    'Shaving & Hair Removal Products': 'Beauty',
    'Fabric Decorating': 'Arts & Crafts',
    'Industrial Materials': 'Industrial',
    'Smart Home: Security Cameras and Systems': 'Smart Home',
    'Office Electronics': 'Electronics',
    'Sports & Outdoor Play Toys': 'Toys',
    "Kids' Play Tractors": 'Toys',
    'Slot Cars, Race Tracks & Accessories': 'Toys',
    'Video Games': 'Electronics',
    'Smart Home: Voice Assistants and Hubs': 'Smart Home',
    'Light Bulbs': 'Home',
    'Toys & Games': 'Toys',
    "Kids' Furniture": 'Home',
    'Automotive Tires & Wheels': 'Automotive',
    'Wellness & Relaxation Products': 'Health',
    'Automotive Tools & Equipment': 'Automotive',
    'Baby & Toddler Toys': 'Baby',
    "Kids' Play Boats": 'Toys',
    'Computer Monitors': 'Electronics',
    "Girls' Jewelry": 'Clothing',
    'Travel': 'Travel',
    'Printmaking Supplies': 'Arts & Crafts',
    "Women's Handbags": 'Clothing',
    'Foot, Hand & Nail Care Products': 'Beauty',
    'Baby & Toddler Feeding Supplies': 'Baby',
    'Computers': 'Electronics',
    'Home Décor Products': 'Home',
    'Industrial Hardware': 'Industrial',
    'Automotive Exterior Accessories': 'Automotive',
    'Skin Care Products': 'Beauty',
    'Wearable Technology': 'Electronics',
    'Reptiles & Amphibian Supplies': 'Pet Supplies',
    'Smart Home: Lawn and Garden': 'Smart Home',
    'Horse Supplies': 'Pet Supplies',
    "Kids' Party Supplies": 'Party Supplies',
    'Tablet Replacement Parts': 'Electronics',
    'Baby Care Products': 'Baby',
    "Kids' Electronics": 'Electronics',
    'Beading & Jewelry Making': 'Arts & Crafts',
    'Computer External Components': 'Electronics',
    'Furniture': 'Home',
    'Nintendo Switch Consoles, Games & Accessories': 'Electronics',
    'Party Decorations': 'Party Supplies',
    'Accessories & Supplies': 'Electronics',
    'Industrial Power & Hand Tools': 'Industrial',
    'Sexual Wellness Products': 'Health',
    'Perfumes & Fragrances': 'Beauty',
    'Household Cleaning Supplies': 'Home',
    "Kids' Play Cars & Race Cars": 'Toys',
    'Pet Bird Supplies': 'Pet Supplies',
    'Janitorial & Sanitation Supplies': 'Industrial',
    'Electrical Equipment': 'Industrial',
    'Dolls & Accessories': 'Toys',
    'RV Parts & Accessories': 'Automotive',
    "Kids' Play Buses": 'Toys',
    'Health & Household': 'Health',
    'Baby Stationery': 'Baby',
    "Girls' School Uniforms": 'Clothing',
    'Travel Duffel Bags': 'Travel',
    'Electronic Components': 'Electronics',
    'Mac Games & Accessories': 'Electronics',
    'Bath Products': 'Beauty',
    'Security & Surveillance Equipment': 'Electronics',
    'Bedding': 'Home',
    'Baby & Child Care Products': 'Baby',
    'Computers & Tablets': 'Electronics',
    'Smart Home: Smart Locks and Entry': 'Smart Devices',
    'Smart Home: WiFi and Networking': 'Smart Devices',
    'Smart Home: Lighting': 'Smart Home',
    'Portable Audio & Video': 'Electronics',
    'Lighting & Ceiling Fans': 'Home',
    'PC Games & Accessories': 'Electronics',
    'Wall Art': 'Home',
    "Baby Girls' Clothing & Shoes": 'Baby',
    'Vehicle Electronics': 'Automotive',
    'Oral Care Products': 'Health',
    'Needlework Supplies': 'Arts & Crafts',
    'Games & Accessories': 'Electronics',
    "Girls' Watches": 'Clothing',
    "Girls' Shoes": 'Clothing',
    'Hair Care Products': 'Beauty',
    'Packaging & Shipping Supplies': 'Industrial',
    "Kids' Dress Up & Pretend Play": 'Toys',
    'Gift Wrapping Supplies': 'Party Supplies',
    'Virtual Reality Hardware & Accessories': 'Electronics',
    'Health Care Products': 'Health',
    'Finger Toys': 'Toys',
    "Kids' Play Trains & Trams": 'Toys',
    "Baby Boys' Clothing & Shoes": 'Baby',
    'Home Appliances': 'Home Appliances',
    'Tricycles, Scooters & Wagons': 'Toys',
    'Fasteners': 'Industrial',
    'Video Projectors': 'Electronics',
    'Vision Products': 'Health',
    'Arts, Crafts & Sewing Storage': 'Arts & Crafts',
    'Science Education Supplies': 'Education',
    'Camera & Photo': 'Electronics',
    'Home Storage & Organization': 'Home',
    'Safety & Security': 'Industrial',
    'Data Storage': 'Electronics',
    'Material Handling Products': 'Industrial',
    'eBook Readers & Accessories': 'Electronics',
    'Baby Strollers & Accessories': 'Baby',
    'Seasonal Décor': 'Home',
    'Heating, Cooling & Air Quality': 'Home Appliances',
    'Lab & Scientific Products': 'Industrial',
    'Smart Home: Other Solutions': 'Smart Devices',
    'Smart Home - Heating & Cooling': 'Smart Devices',
    'Sports & Fitness': 'Sports',
    'Building Supplies': 'Tools & Home Improvement',
    'Building Toys': 'Toys',
    'Xbox One Games, Consoles & Accessories': 'Electronics',
    "Men's Watches": 'Clothing',
    'Novelty Toys & Amusements': 'Toys',
    'Cutting Tools': 'Tools & Home Improvement',
    'Laptop Accessories': 'Electronics',
    'Industrial & Scientific': 'Industrial',
    'Household Supplies': 'Home',
    "Boys' Jewelry": 'Clothing',
    'Filtration': 'Industrial',
    'Small Animal Supplies': 'Pet Supplies',
    'Toy Figures & Playsets': 'Toys',
    'PlayStation 3 Games, Consoles & Accessories': 'Electronics',
    'Xbox Series X & S Consoles, Games & Accessories': 'Electronics',
    'Smart Home: Home Entertainment': 'Smart Home',
    'Professional Dental Supplies': 'Industrial',
    "Women's Accessories": 'Clothing',
    'Heavy Duty & Commercial Vehicle Equipment': 'Automotive',
    'Computer Components': 'Electronics',
    'Baby Activity & Entertainment Products': 'Baby',
    'Painting, Drawing & Art Supplies': 'Arts & Crafts',
    'Lights, Bulbs & Indicators': 'Automotive',
    'Welding & Soldering': 'Industrial',
    'Home Use Medical Supplies & Equipment': 'Health',
    "Women's Clothing": 'Clothing',
    'Knitting & Crochet Supplies': 'Arts & Crafts',
    'Commercial Door Products': 'Industrial',
    'Automotive Enthusiast Merchandise': 'Automotive',
    'Sports Nutrition Products': 'Health',
    'Beauty & Personal Care': 'Beauty',
    'Makeup': 'Beauty',
    'Baby': 'Baby',
    'Learning & Education Toys': 'Toys',
    'GPS & Navigation': 'Electronics',
    'Motorcycle & Powersports': 'Automotive',
    'Video Game Consoles & Accessories': 'Electronics',
    "Boys' School Uniforms": 'Clothing',
    'Online Video Game Services': 'Electronics',
    'PlayStation 5 Consoles, Games & Accessories': 'Electronics',
    'Smart Home: Plugs and Outlets': 'Smart Home',
    'Smart Home: Vacuums and Mops': 'Smart Home',
    'Outdoor Recreation': 'Sports & Outdoors',
    'Sony PSP Games, Consoles & Accessories': 'Electronics',
    'Sports & Outdoors': 'Sports & Outdoors',
    'Kitchen & Bath Fixtures': 'Home',
    'Gift Cards': 'Gift Cards',   # manually updated to match instead of Other
    "Women's Jewelry": 'Clothing',
    'Oils & Fluids': 'Automotive',
    'Toilet Training Products': 'Baby',
    'Baby Safety Products': 'Baby',
    'Messenger Bags': 'Travel',
    "Boys' Accessories": 'Clothing',
    'Garment Bags': 'Travel',
    'Nursery Furniture, Bedding & Décor': 'Baby',
    'Power Transmission Products': 'Industrial',
    'Kitchen & Dining': 'Home & Kitchen',
    'Beauty Tools & Accessories': 'Beauty',
    'Computer Servers': 'Electronics',
    'Hydraulics, Pneumatics & Plumbing': 'Industrial',
    'Party Supplies': 'Party Supplies',
    'Dog Supplies': 'Pet Supplies',
    'Occupational Health & Safety Products': 'Industrial',
    'Cell Phones & Accessories': 'Electronics',
    'Craft & Hobby Fabric': 'Arts & Crafts',
    'Child Safety Car Seats & Accessories': 'Baby',
    'Diet & Sports Nutrition': 'Health',
    'Industrial Adhesives, Sealants & Lubricants': 'Industrial',
    "Kids' Home Store": 'Home',
    'Cat Supplies': 'Pet Supplies',
    'Hardware': 'Tools & Home Improvement',
    'Arts & Crafts Supplies': 'Arts & Crafts',
    "Women's Watches": 'Clothing',
    'Professional Medical Supplies': 'Industrial',
    'Toy Vehicle Playsets': 'Toys',
    'Sewing Products': 'Arts & Crafts',
    'Abrasive & Finishing Products': 'Industrial',
    'Computer Networking': 'Electronics',
    'Home Lighting & Ceiling Fans': 'Home',
    'Ironing Products': 'Home',
    'Retail Store Fixtures & Equipment': 'Industrial',
    'Home Audio & Theater Products': 'Electronics',
    "Boys' Shoes": 'Clothing',
    'Tools & Home Improvement': 'Tools & Home Improvement',
    'Fish & Aquatic Pets': 'Pet Supplies',
    'Pumps & Plumbing Equipment': 'Industrial',
    'Nintendo DS Games, Consoles & Accessories': 'Electronics',
    'Wii Games, Consoles & Accessories': 'Electronics',
    'Power Tools & Hand Tools': 'Tools & Home Improvement',
    'Smart Home Thermostats - Compatibility Checker': 'Smart Home',
    "Women's Shoes": 'Clothing',
    'Car Care': 'Automotive',
    'Baby Travel Gear': 'Baby',
    'Baby Gifts': 'Baby',
    'Luggage Sets': 'Travel',
    'Automotive Paint & Paint Supplies': 'Automotive',
    'Personal Care Products': 'Beauty',
    'Baby Diapering Products': 'Baby',
    'Puzzles': 'Toys',
    'Nintendo 3DS & 2DS Consoles, Games & Accessories': 'Electronics',
    'Legacy Systems': 'Electronics',
    'Measuring & Layout': 'Tools & Home Improvement',
    'Smart Home: New Smart Devices': 'Smart Home',
    'Stuffed Animals & Plush Toys': 'Toys',
    "Kids' Play Trucks": 'Toys',
    'Craft Supplies & Materials': 'Arts & Crafts',
    'Automotive Performance Parts & Accessories': 'Automotive',
    'Automotive Replacement Parts': 'Automotive',
    'Puppets & Puppet Theaters': 'Toys',
    'Tablet Accessories': 'Electronics',
    "Girls' Accessories": 'Clothing',
    'Laptop Bags': 'Travel',
    'Backpacks': 'Travel',
    'Scrapbooking & Stamping Supplies': 'Arts & Crafts',
    'Food Service Equipment & Supplies': 'Industrial',
    'Test, Measure & Inspect': 'Industrial',
    'Travel Tote Bags': 'Travel',
    'Automotive Interior Accessories': 'Automotive',
    'Paint, Wall Treatments & Supplies': 'Tools & Home Improvement',
    'Rain Umbrellas': 'Travel',
    'Travel Accessories': 'Travel',
    'Stationery & Gift Wrapping Supplies': 'Party Supplies',
    'Car Electronics & Accessories': 'Automotive',
}

# Use categories_dict results to create a new df column
merged['broad_category'] = merged['category_name'].map(categories_dict)

# Check
merged.head()

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,isBestSeller,boughtInLastMonth,category_name,broad_category
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.50,0,139.99,0.00,False,2000,Suitcases,Travel
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.50,0,169.99,209.99,False,1000,Suitcases,Travel
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,https://www.amazon.com/dp/B07XSCCZYG,4.60,0,365.49,429.99,False,300,Suitcases,Travel
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,https://www.amazon.com/dp/B08MVFKGJM,4.60,0,291.59,354.37,False,400,Suitcases,Travel
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,https://www.amazon.com/dp/B01DJLKZBA,4.50,0,174.99,309.99,False,400,Suitcases,Travel


## Create Final Dataset .csv and .json

In [14]:
# Save to final .csv
merged.to_csv("data/final_products.csv", index=False, encoding="utf-8")

# Save to final JSON for SE/UI/UX
merged.to_json("data/final_products.json", orient="records", lines=True)

## Remove data no longer sold, missing image links, etc.?

Not sure if needed.

In [ ]:
# 2000 product pilot
import pandas as pd
import requests
import time

# -----------------------
# CONFIG
# -----------------------
CSV_PATH = "data/final_products.csv"
SAMPLE_SIZE = 2000        # small pilot
TIMEOUT = 5               # seconds
USER_AGENT = "Mozilla/5.0"

UNAVAILABLE_PHRASES = [
    "currently unavailable",
    "we don't know when or if this item will be back",
    "couldn't find that page",
    "looking for something?"
]

# -----------------------
# SETUP
# -----------------------
session = requests.Session()
HEADERS = {"User-Agent": USER_AGENT}

# -----------------------
# CHECK FUNCTIONS
# -----------------------
def check_image(url):
    try:
        r = session.head(url, timeout=TIMEOUT)
        return r.status_code == 200
    except requests.exceptions.RequestException:
        return False

def check_product(url):
    start = time.time()
    try:
        # Fast fail
        r = session.head(url, headers=HEADERS, timeout=TIMEOUT, allow_redirects=True)
        if r.status_code in [404, 410]:
            return "dead", time.time() - start

        # Fallback to GET
        r = session.get(url, headers=HEADERS, timeout=TIMEOUT)
        text = r.text.lower()

        if any(p in text for p in UNAVAILABLE_PHRASES):
            return "unavailable", time.time() - start

        return "available", time.time() - start

    except requests.exceptions.RequestException:
        return "error", time.time() - start

# -----------------------
# LOAD + SAMPLE
# -----------------------
df = pd.read_csv(CSV_PATH)

sample_df = df.sample(
    n=min(SAMPLE_SIZE, len(df)),
    random_state=42
).reset_index(drop=True)

print(f"Running pilot on {len(sample_df)} products...")

# -----------------------
# IMAGE CHECK (FAST SIGNAL)
# -----------------------
t0 = time.time()
sample_df["image_alive"] = sample_df["imgUrl"].apply(check_image)
print(f"Image check completed in {time.time() - t0:.1f} sec")

# -----------------------
# PRODUCT PAGE CHECK
# -----------------------
results = sample_df["productURL"].apply(check_product)
sample_df["status"] = results.apply(lambda x: x[0])
sample_df["latency_sec"] = results.apply(lambda x: x[1])

# -----------------------
# SUMMARY
# -----------------------
print("\nStatus distribution:")
print(sample_df["status"].value_counts(normalize=True).round(3))

print("\nLatency (seconds):")
print(sample_df["latency_sec"].describe().round(2))

print("\nImage vs status:")
print(pd.crosstab(sample_df["status"], sample_df["image_alive"]))

# -----------------------
# SAVE RESULTS
# -----------------------
sample_df.to_csv("data/amazon_availability.csv", index=False)
print("\nSaved results to amazon_availability_pilot.csv")


Running pilot on 2000 products...
Image check completed in 324.8 sec

Status distribution:
status
available     0.90
unavailable   0.09
error         0.01
Name: proportion, dtype: float64

Latency (seconds):
count   2000.00
mean       0.24
std        0.44
min        0.11
25%        0.12
50%        0.12
75%        0.17
max        5.31
Name: latency_sec, dtype: float64

Image vs status:
image_alive  False  True 
status                   
available       11   1787
error            0     15
unavailable      1    186

Saved results to amazon_availability_pilot.csv


In [ ]:
# 2000 product pilot with async
import pandas as pd
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# -----------------------
# CONFIG
# -----------------------
CSV_PATH = "data/final_products.csv"
SAMPLE_SIZE = 2000
TIMEOUT = 5
MAX_WORKERS = 8

HEADERS = {"User-Agent": "Mozilla/5.0"}

UNAVAILABLE_PHRASES = [
    "currently unavailable",
    "we don't know when or if this item will be back",
    "couldn't find that page",
    "looking for something?"
]

session = requests.Session()

# -----------------------
# CHECK FUNCTIONS
# -----------------------
def image_alive(url):
    try:
        r = session.head(url, timeout=TIMEOUT)
        return r.status_code == 200
    except:
        return False

def check_product(url):
    try:
        r = session.head(url, headers=HEADERS, timeout=TIMEOUT, allow_redirects=True)

        if r.status_code in [404, 410]:
            return "dead"

        r = session.get(url, headers=HEADERS, timeout=TIMEOUT)
        text = r.text.lower()

        if any(p in text for p in UNAVAILABLE_PHRASES):
            return "unavailable"

        return "available"

    except:
        return "error"

def process_row(row):
    start = time.time()

    img_ok = image_alive(row["imgUrl"])
    if not img_ok:
        return {
            "status": "dead_image",
            "latency_sec": time.time() - start,
            "image_alive": False
        }

    status = check_product(row["productURL"])

    return {
        "status": status,
        "latency_sec": time.time() - start,
        "image_alive": True
    }

# -----------------------
# LOAD + SAMPLE
# -----------------------
df = pd.read_csv(CSV_PATH)

sample_df = df.sample(
    n=min(SAMPLE_SIZE, len(df)),
    random_state=42
).reset_index(drop=True)

print(f"Running optimized pilot on {len(sample_df)} products...")

# -----------------------
# RUN CONCURRENTLY
# -----------------------
t0 = time.time()
results = []

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {
        executor.submit(process_row, row): idx
        for idx, row in sample_df.iterrows()
    }

    for future in as_completed(futures):
        results.append(future.result())

elapsed = time.time() - t0

# -----------------------
# MERGE RESULTS
# -----------------------
results_df = pd.concat(
    [sample_df.reset_index(drop=True), pd.DataFrame(results)],
    axis=1
)

# -----------------------
# SUMMARY
# -----------------------
print(f"\nTotal runtime: {elapsed:.1f} sec")
print(f"Avg per item: {elapsed / len(results_df):.3f} sec")

print("\nStatus distribution:")
print(results_df["status"].value_counts(normalize=True).round(3))

print("\nLatency (seconds):")
print(results_df["latency_sec"].describe().round(2))

print("\nImage short-circuit rate:")
print(results_df["image_alive"].value_counts(normalize=True).round(3))

# -----------------------
# SAVE
# -----------------------
results_df.to_csv("data/amazon_availability_pilot_fast.csv", index=False)
print("\nSaved to amazon_availability_pilot_fast.csv")

Running optimized pilot on 2000 products...

Total runtime: 98.9 sec
Avg per item: 0.049 sec

Status distribution:
status
available     0.88
unavailable   0.11
error         0.01
dead_image    0.00
Name: proportion, dtype: float64

Latency (seconds):
count   2000.00
mean       0.39
std        0.58
min        0.13
25%        0.19
50%        0.24
75%        0.32
max        5.44
Name: latency_sec, dtype: float64

Image short-circuit rate:
image_alive
True    1.00
False   0.00
Name: proportion, dtype: float64

Saved to amazon_availability_pilot_fast.csv


In [ ]:
# Final availability filter
import pandas as pd
import asyncio
import aiohttp
import async_timeout
from aiohttp import ClientSession
import time
import nest_asyncio

# -----------------------
# CONFIG
# -----------------------
CSV_PATH = "data/final_products.csv"
OUTPUT_CSV = "data/amazon_availability_final.csv"
SAMPLE_SIZE = None  # set to an int for small test, or None for full dataset
CONCURRENT_REQUESTS = 30
TIMEOUT = 8
USER_AGENT = "Mozilla/5.0"

UNAVAILABLE_PHRASES = [
    "currently unavailable",
    "we don't know when or if this item will be back",
    "couldn't find that page",
    "looking for something?"
]

HEADERS = {"User-Agent": USER_AGENT}

# -----------------------
# LOAD DATA
# -----------------------
df = pd.read_csv(CSV_PATH)
if SAMPLE_SIZE:
    df = df.sample(n=SAMPLE_SIZE, random_state=42).reset_index(drop=True)
else:
    df = df.reset_index(drop=True)

# -----------------------
# ASYNC CHECK FUNCTIONS
# -----------------------
async def fetch_head(session, url):
    try:
        async with async_timeout.timeout(TIMEOUT):
            async with session.head(url, headers=HEADERS, allow_redirects=True) as resp:
                return resp.status
    except:
        return None

async def fetch_get(session, url):
    try:
        async with async_timeout.timeout(TIMEOUT):
            async with session.get(url, headers=HEADERS) as resp:
                text = await resp.text()
                return text.lower()
    except:
        return None

async def check_product(session, row):
    # First, check image
    try:
        async with async_timeout.timeout(TIMEOUT):
            async with session.head(row["imgUrl"]) as img_resp:
                image_alive = img_resp.status == 200
    except:
        image_alive = False

    if not image_alive:
        return "unavailable"

    # Check product page HEAD
    head_status = await fetch_head(session, row["productURL"])
    if head_status in [404, 410]:
        return "unavailable"

    # GET page to look for unavailable text
    page_text = await fetch_get(session, row["productURL"])
    if page_text is None:
        return "unavailable"

    if any(p in page_text for p in UNAVAILABLE_PHRASES):
        return "unavailable"

    return "available"

# -----------------------
# ASYNC MAIN LOOP
# -----------------------
async def main():
    connector = aiohttp.TCPConnector(limit=CONCURRENT_REQUESTS)
    timeout = aiohttp.ClientTimeout(total=TIMEOUT+2)
    async with ClientSession(connector=connector, timeout=timeout) as session:
        tasks = []
        results = [None] * len(df)

        semaphore = asyncio.Semaphore(CONCURRENT_REQUESTS)

        async def sem_task(idx, row):
            async with semaphore:
                status = await check_product(session, row)
                results[idx] = status

        for idx, row in df.iterrows():
            tasks.append(asyncio.create_task(sem_task(idx, row)))

        await asyncio.gather(*tasks)

    df["availability"] = results
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"Done! Saved results to {OUTPUT_CSV}")

# -----------------------
# RUN
# -----------------------
nest_asyncio.apply()  # allows nested event loops

start_time = time.time()
await main()  # use await instead of asyncio.run()
print(f"Elapsed time: {time.time() - start_time:.1f} sec")

In [21]:
merged.head(20)

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,isBestSeller,boughtInLastMonth,category_name,broad_category
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.50,0,139.99,0.00,False,2000,Suitcases,Travel
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.50,0,169.99,209.99,False,1000,Suitcases,Travel
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,https://www.amazon.com/dp/B07XSCCZYG,4.60,0,365.49,429.99,False,300,Suitcases,Travel
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,https://www.amazon.com/dp/B08MVFKGJM,4.60,0,291.59,354.37,False,400,Suitcases,Travel
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,https://www.amazon.com/dp/B01DJLKZBA,4.50,0,174.99,309.99,False,400,Suitcases,Travel
5,B07XSCD2R4,Maxlite 5 Softside Expandable Luggage with 4 S...,https://m.media-amazon.com/images/I/61LnBNsSBS...,https://www.amazon.com/dp/B07XSCD2R4,4.50,0,144.49,0.00,False,500,Suitcases,Travel
6,B07MXF4G8K,"Hard Shell Carry on Luggage Airline Approved, ...",https://m.media-amazon.com/images/I/71CghLYrnA...,https://www.amazon.com/dp/B07MXF4G8K,4.50,0,169.99,0.00,False,400,Suitcases,Travel
7,B07H515VCZ,"Maxporter II 30"" Hardside Spinner Trunk Luggag...",https://m.media-amazon.com/images/I/81f3h+YHOX...,https://www.amazon.com/dp/B07H515VCZ,4.50,0,299.99,0.00,False,100,Suitcases,Travel
8,B08BXBCNMQ,Omni 2 Hardside Expandable Luggage with Spinne...,https://m.media-amazon.com/images/I/91eOWP4myS...,https://www.amazon.com/dp/B08BXBCNMQ,4.50,0,112.63,137.04,False,500,Suitcases,Travel
9,B0B9K44XTS,Luggage Sets Expandable Lightweight Suitcases ...,https://m.media-amazon.com/images/I/81dsv5GrCL...,https://www.amazon.com/dp/B0B9K44XTS,4.40,0,209.99,0.00,False,200,Suitcases,Travel


## Embeddings

## Visualizations for Dashboard

### # Items per Category

In [23]:
category_counts = merged['broad_category'].value_counts().reset_index()

fig = px.bar(category_counts, 
             x='broad_category',
             y='count',
             color='broad_category',
             title="Products per Category")
#fig.show()
fig.write_html("plots/items_per_broad_category.html")

### Price Distribution

In [24]:
max_price = 300
filtered = merged[merged['price'] <= max_price]

fig = px.histogram(filtered, 
                   x='price', 
                   nbins=100, 
                   title=f"Price Distribution (<= ${max_price})")
#fig.show()
fig.write_html("plots/price_distribution.html")

### Rating vs. Reviews

In [25]:
fig = px.scatter(
    merged,
    x='stars',
    y='reviews',
    size='reviews',
    color='broad_category',
    hover_data=['title', 'price', 'stars', 'reviews'],
    title="Product Ratings vs Review Count",
    size_max=30
)

fig.update_layout(yaxis_type='log')  # optional, if reviews range widely
#fig.show()
fig.write_html("plots/rating_vs_reviews_broad_category.html")